# Main Result
This version removes validation samples from sites with no training data from the within-site results

## Initialisation

In [ ]:
import glob
import json
import numpy as np
import os
import pandas as pd
import seaborn as sns

from collections import defaultdict

import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerTuple, HandlerPatch
import matplotlib.text as mpl_text
from matplotlib.patches import ConnectionPatch, Rectangle

import initialise
import common
from analysis_utils import calc_statistics, grouped_results
from display_utils import display_frames

In [ ]:
%matplotlib inline

In [ ]:
samples_file = os.path.join(common.DATASETS_DIR, 'samples_730days.csv')
model_dir = os.path.join(common.MODELS_DIR, 'evaluation_models')

In [ ]:
model = common.ANALYSIS_MODEL
ensemble_size = common.ENSEMBLE_SIZE
ensemble_runs = common.ENSEMBLE_RUNS
precision = 2

## Get the samples and predictions

In [ ]:
def sort_file_list(file_list, basename_prefix):
    return sorted(
        file_list,
        key=lambda x: int(os.path.splitext(x)[0].rsplit(basename_prefix, 1)[1] or 0)
        )

In [ ]:
stats_list = []
test_list = []
num_samples = []
num_predicts = []
samples = pd.read_csv(samples_file, index_col=0)
with open(os.path.join(model_dir, 'experiment.json'), 'r') as f:
    experiment = json.load(f)
test_dirs = sort_file_list(glob.glob(os.path.join(model_dir, f'test*')), 'test')
for test_num, test_dir in enumerate(test_dirs):
    stats_ = pd.read_csv(os.path.join(test_dir, f'ensemble{ensemble_size}_stats.csv'), index_col=(0,1))
    stats_list.append(stats_.loc[model].mean(axis=1))
    try:
        tn = experiment['tests'][test_num].get('testName') or experiment['testNames'][test_num]
    except:
        tn = f'Test {test_num}'
    test_list.append(tn)
    num_samples.append(samples.shape[0])

In [ ]:
stats_df = pd.DataFrame(stats_list, index=test_list)
stats_df['#Samples'] = num_samples
stats_df.round(3)

In [ ]:
with open(os.path.join(model_dir, 'model_params.json'), 'r') as f:
    params = json.load(f)
predicts = []
for test_num, test_dir in enumerate(test_dirs):
    predicts.append(pd.read_csv(os.path.join(test_dir, f'ensemble{ensemble_size}_{model}.csv'), index_col=0))

In [ ]:
samples1 = samples.reindex(predicts[0].index)
yearly_summ = samples1.groupby(['Sampling year']).agg(**{
    "Sample count": pd.NamedAgg(column=params['targetColumn'], aggfunc="count"),
    "LFMC minimum": pd.NamedAgg(column=params['targetColumn'], aggfunc="min"),
    "LFMC median": pd.NamedAgg(column=params['targetColumn'], aggfunc="median"),
    "LFMC mean": pd.NamedAgg(column=params['targetColumn'], aggfunc="mean"),
    "LFMC maximum": pd.NamedAgg(column=params['targetColumn'], aggfunc="max"),
    "LFMC std": pd.NamedAgg(column=params['targetColumn'], aggfunc="std"),
}).round(2)

rmse_df = grouped_results(samples1, predicts, 'Sampling year', params['targetColumn'], test_list, measures='RMSE')
rmse_df = pd.concat([stats_df.RMSE, rmse_df], axis=1)
rmse_df.rename(columns={'RMSE': 'All'}, inplace=True)

r2_df = grouped_results(samples1, predicts, 'Sampling year', params['targetColumn'], test_list, measures='R2')
r2_df = pd.concat([stats_df.R2, r2_df], axis=1)
r2_df.rename(columns={'R2': 'All'}, inplace=True)

bias_df = grouped_results(samples1, predicts, 'Sampling year', params['targetColumn'], test_list, measures='Bias')
bias_df = pd.concat([stats_df.Bias, bias_df], axis=1)
bias_df.rename(columns={'Bias': 'All'}, inplace=True)

In [ ]:
summ_stats = [samples[params['targetColumn']].describe(), samples1[params['targetColumn']].describe()]
for test_ in predicts:
    summ_stats.append(test_.mean(axis=1).describe())
summ_stats = pd.DataFrame(summ_stats, index=['All samples', 'Evaluation samples'] + test_list,
                          columns=['count', 'min', '50%', 'mean', 'max', 'std']).round(1)
summ_stats.columns = ['count', 'min', 'median', 'mean', 'max', 'std']
summ_stats

In [ ]:
print("Number of samples with LFMC <= 40:",
      samples1[samples1[params['targetColumn']] <= 40][params['targetColumn']].count())
print("Number of samples with LFMC >= 255:",
      samples1[samples1[params['targetColumn']] >= 255][params['targetColumn']].count())
print("Kurtosis of all evaluation samples:",
      samples1[params['targetColumn']].kurt().round(2))
print("Kurtosis where LFMC > 40:",
      samples1[samples1[params['targetColumn']] > 40][params['targetColumn']].kurt().round(2))
print("Kurtosis where LFMC < 255:",
      samples1[samples1[params['targetColumn']] < 255][params['targetColumn']].kurt().round(2))
print("Kurtosis where LFMC between 40 and 255:",
      samples1[(samples1[params['targetColumn']] > 40) & (samples1[params['targetColumn']] < 255)]
              [params['targetColumn']].kurt().round(2))
print("Therefore, excess kurtosis appears to be due to the samples with very high LFMC.")

In [ ]:
class AnyObject(object):
    def __init__(self, text1, text2):
        self.text1 = text1
        self.text2 = text2

In [ ]:
class AnyObjectHandler(object):
    def legend_artist(self, legend, text, fontsize, handlebox):
        patch1 = mpl_text.Text(x=0, y=0, text=f'{text.text1} {text.text2}', size=5)
        handlebox.add_artist(patch1)
        return patch1

In [ ]:
palette = [sns.color_palette()[0], sns.color_palette()[2], sns.color_palette()[1], sns.color_palette()[4], sns.color_palette()[5]]
ls = ['-', '-.', '--', ':', (0, (4, 3, 1, 3, 1, 3))]
marker = ['X', 'd', 'o', '*', 'P']
cycler = plt.cycler(linestyle=ls, color=palette, marker=marker)
font_size = 7
title_size = 8
plt.rcParams.update({'font.size': font_size})
plt.rcParams.update({'font.sans-serif': 'Arial'})

In [ ]:
def plot_summ_stats(summ_stats, ax, xlim=[None], ylim=[None], legend=True):
    df = summ_stats.iloc[2:]
    ax.set_box_aspect(1)
    ax.set_prop_cycle(cycler)
    ax = df[['max', 'mean', 'median', 'min']].plot.line(ax=ax, linewidth=1, markersize=4)
    fill1 = ax.fill_between(
        np.arange(13),
        df['mean'] - df['std'],  #stats_plot['iqr_lower'],
        df['mean'] + df['std'],  #stats_plot['iqr_upper'],
        color=palette[1],
        alpha=0.3, lw=0
    )

    mn = summ_stats.loc['Evaluation samples', 'mean']
    sd = summ_stats.loc['Evaluation samples', 'std']
    p5 = ax.plot([-1, -1], [mn-sd, mn+sd], color=palette[1], alpha=0.3, ls='-', lw=2, marker="None")
    p1 = ax.plot(-1, summ_stats.loc['Evaluation samples', 'max'], marker[0], markeredgewidth=0.7, markersize=4, markeredgecolor=palette[0], markerfacecolor='w')
    p2 = ax.plot(-1, summ_stats.loc['Evaluation samples', 'mean'], marker[1], markeredgewidth=0.7, markersize=4, markeredgecolor=palette[1], markerfacecolor='None')
    p3 = ax.plot(-1, summ_stats.loc['Evaluation samples', 'median'], marker[2], markeredgewidth=0.7, markersize=4, markeredgecolor=palette[2], markerfacecolor='None')
    p4 = ax.plot(-1, summ_stats.loc['Evaluation samples', 'min'], marker[3], markeredgewidth=0.7, markersize=4, markeredgecolor=palette[3], markerfacecolor='w')

    ax.set_xlabel('Lead time (months)', size=7)
    ax.set_xticks(np.arange(13))
    ax.set_xticklabels(np.arange(13), fontsize=7)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.tick_params(labelsize=7)
    
    # Add the legend
    if legend:
        handles, labels = ax.get_legend_handles_labels()
        obj_0 = AnyObject("Measured", "Estimated")
        temp_leg = ax.legend([obj_0] + list(zip([p1[0], p2[0], p3[0], p4[0]], handles)), [''] + labels,
                             handler_map={obj_0:AnyObjectHandler(), tuple: HandlerTuple(ndivide=None)},
                             fontsize=7, handlelength=6, labelspacing=0.3, borderpad=0.3, handletextpad=0.3)
    else:
        ax.legend().remove()

In [ ]:
def add_annotation(ax, text, point, text_point):
    text_size = 5
    text_box = dict(boxstyle="square, pad=0", lw=0, fc='none')
    arrow = {'arrowstyle': '->'}
    ax.annotate(text, point, text_point, textcoords='offset points', size=text_size, arrowprops=arrow, bbox=text_box)

In [ ]:
summ_dfs = []
for year in [2014, 2015, 2016, 2017]:
    df_ = [samples1[samples1['Sampling year'] == year]['LFMC value'].describe()]
    for test_ in predicts:
        df_.append(test_[samples1['Sampling year'] == year].mean(axis=1).describe())
    df_ = pd.DataFrame(df_, index=['Measured'] + test_list, columns=['count', 'min', '50%', 'mean', 'max', 'std']).round(1)
    df_.columns = ['count', 'min', 'median', 'mean', 'max', 'std']
    summ_dfs.append(df_)
display_frames(summ_dfs, ['2014', '2015', '2016', '2017'], precision=1)

In [ ]:
temp_df = summ_stats.loc[['Nowcasting', '3-months lead time', 'Evaluation samples']].rename(index={'Evaluation samples': 'Measured'})
summ_df = pd.concat([temp_df] + [s.loc[['Nowcasting', '3-months lead time', 'Measured']] for s in summ_dfs],
                    keys=['All years', '2014', '2015', '2016', '2017'])
bp_data = [samples1[['Sampling year', 'LFMC value']].rename(columns={'LFMC value': 'LFMC (%)'})]
bp_data.append(pd.concat([samples1['Sampling year'], predicts[0].mean(axis=1)], axis=1).rename(columns={0: 'LFMC (%)'}))
bp_data.append(pd.concat([samples1['Sampling year'], predicts[3].mean(axis=1)], axis=1).rename(columns={0: 'LFMC (%)'}))
bp_data = pd.concat(bp_data, keys=['Nowcast estimate', '3-month projection', 'Measured LFMC']).reset_index()
bp_data1 = bp_data.copy()
bp_data1['Sampling year'] = 'All years'
bp_data = pd.concat([bp_data, bp_data1])

In [ ]:
display_frames([rmse_df, r2_df, bias_df],
               ['RMSE', 'R2', 'Bias'],
               precision=2)

In [ ]:
palette = [sns.color_palette()[0], sns.color_palette()[1], sns.color_palette()[2], sns.color_palette()[4], sns.color_palette()[5]]
ls = ['-', '-.', '--', ':', (0, (4, 3, 1, 3, 1, 3))]
cycler = plt.cycler(linestyle=ls, color=palette, marker=['X', 'o', 'd', '*', 'P'])
font_size = 7
title_size = 8
plt.rcParams.update({'font.size': font_size})
plt.rcParams.update({'font.sans-serif': 'Arial'})
fig, ((ax1, ax4), (ax2, ax5), (ax3, ax6)) = plt.subplots(3, 2, figsize=(5, 6), gridspec_kw={'width_ratios': [3, 1]},
    sharex=True, sharey=False, constrained_layout=True, dpi=600, linewidth=2, edgecolor="black")

ax1.set_prop_cycle(cycler)
ax1 = rmse_df.plot.line(ax=ax1, linewidth=1, markersize=4)
ax1.set_xlabel('Lead time (months)', size=title_size)
ax1.set_ylabel('RMSE (%)', size=title_size)
ax1.set_xticks(np.arange(13))
ax1.set_xticklabels(np.arange(13), fontsize=font_size)
ax1.tick_params(labelsize=font_size)
ax1.text(-1.8, 29.9, '(a)', va='top', ha='right', fontsize=title_size)

ax2.set_prop_cycle(cycler)
ax2 = r2_df.plot(ax=ax2, linewidth=1, markersize=4)
ax2.set_xlabel('Lead time (months)', size=title_size)
ax2.set_ylabel('$R^2$', size=title_size)
ax2.set_xticks(np.arange(13))
ax2.set_xticklabels(np.arange(13))
ax2.tick_params(labelsize=font_size)
ax2.set_ylim((0.3825, 0.5625))
ax2.set_yticks(np.arange(40, 55, 2)/100)
ax2.text(-1.8, 0.5625, '(b)', va='top', ha='right', fontsize=title_size)

ax3.set_prop_cycle(cycler)
ax3 = bias_df.plot(ax=ax3, linewidth=1, markersize=4)
ax3.set_xlabel('Lead time (months)', size=title_size)
ax3.set_ylabel('Bias (%)', size=title_size)
ax3.set_xticks(np.arange(13))
ax3.set_ylim((-4.7, 7.99))
ax3.tick_params(labelsize=font_size)
ax3.set_xticklabels(np.arange(13))
ax3.text(-1.8, 8, '(c)', va='top', ha='right', fontsize=title_size)

lines, labels = ax3.get_legend_handles_labels()
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')
ax1.get_legend().remove()
ax2.get_legend().remove()
ax3.get_legend().remove()
ax4.legend(lines, labels, loc = 'center', title='Legend', fontsize=font_size, handlelength=5, labelspacing=1.5, borderpad=1) #, borderaxespad=0.4)

fig.savefig(os.path.join(common.FIGURES_DIR, 'Lead times.jpeg'), format="jpg", bbox_inches='tight', pad_inches=0.2, dpi=600)

In [ ]:
plt.rcParams['lines.linewidth'] = 0.5
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3.5), dpi=200, linewidth=2, edgecolor="black")

ax = ax1
ax = sns.boxplot(x='LFMC (%)', y='Sampling year', hue='level_0', data=bp_data, showfliers=False, linewidth=1, ax=ax, orient='h',
                 order=[2014, 2015, 2016, 2017, 'All years'],
                 hue_order=['Nowcast estimate', '3-month projection', 'Measured LFMC'])

for xtick in ax.get_yticks():
    year_ = ax.get_yticklabels()[xtick].get_text()
    for pos, label in enumerate(['Nowcasting', '3-months lead time', 'Measured', ]):
        x_off = xtick + [-0.267, 0.0, .267][pos]
        
        # Add the minimums and maximums
        min_ = summ_df.loc[(year_, label), 'min']
        max_ = summ_df.loc[(year_, label), 'max']
        col_ = plt.rcParams["axes.prop_cycle"].by_key()["color"][pos]
        pt, = ax.plot(min_, x_off, 'ok', markersize=5, scalex=False,
                      mfc=col_, mec=col_)
        pt, = ax.plot(max_, x_off, '*k', markersize=7, scalex=False,
                      mfc=col_, mec=col_)

ax.set_ylabel('Evaluation year')
ax.set_yticklabels([2014, 2015, 2016, 2017, 'All  \nyears']) #, ha='center')
ax.legend(loc='upper right', bbox_to_anchor=(0.99, 0.995), labelspacing=0.3, borderpad=0.4, handletextpad=0.3, borderaxespad=0)
ax.set_xlim(-10, 450)

ax = ax2
ax.axis('off')
ax.axis('tight')
tab_font_size = 7
tab_col_widths = [0.1, 0.1, 0.1, 0.1, 0.1]
tab_line_width = 0.5
row_names = dict(zip(['Nowcasting', '3-month lead time', 'Measured'], ['Nowcast estimate', '3-month projection', 'Measured LFMC']))

temp_df = summ_df.loc['2014'].drop(columns='count').round(1).rename(row_names)
count_ = int(summ_df.loc['2014', 'count'][0])
tab1 = ax.table(cellText=temp_df.values, rowLabels=temp_df.index, colLabels=temp_df.columns,
                colWidths=tab_col_widths, bbox=[0.25, .816, 0.75, .184])
width_ = tab1[1, -1].get_width()
height_ = tab1[0, 0].get_height()
tab1.add_cell(0, -1, width=width_, height=height_, text=f'2014 (count: {count_})', fontproperties=dict(size=tab_font_size))
tab1.auto_set_font_size(False)
tab1.set_fontsize(tab_font_size)
for cell_ in tab1.get_children():
    cell_._linewidth = tab_line_width

temp_df = summ_df.loc['2015'].drop(columns='count').rename(row_names)
count_ = int(summ_df.loc['2015', 'count'][0])
tab2 = ax.table(cellText=temp_df.values, rowLabels=temp_df.index, colLabels=temp_df.columns, 
                colWidths=tab_col_widths, bbox=[0.25, .612, 0.75, .184])
width_ = tab2[1, -1].get_width()
height_ = tab2[0, 0].get_height()
tab2.add_cell(0, -1, width=width_, height=height_, text=f'2015 (count: {count_})', fontproperties=dict(size=tab_font_size))
tab2.auto_set_font_size(False)
tab2.set_fontsize(tab_font_size)
for cell_ in tab2.get_children():
    cell_._linewidth = tab_line_width

temp_df = summ_df.loc['2016'].drop(columns='count').rename(row_names)
count_ = int(summ_df.loc['2016', 'count'][0])
tab3 = ax.table(cellText=temp_df.values, rowLabels=temp_df.index, colLabels=temp_df.columns,
                colWidths=tab_col_widths, bbox=[0.25, .408, 0.75, .184])
width_ = tab3[1, -1].get_width()
height_ = tab3[0, 0].get_height()
tab3.add_cell(0, -1, width=width_, height=height_, text=f'2016 (count: {count_})', fontproperties=dict(size=tab_font_size))
tab3.auto_set_font_size(False)
tab3.set_fontsize(tab_font_size)
for cell_ in tab3.get_children():
    cell_._linewidth = tab_line_width

temp_df = summ_df.loc['2017'].drop(columns='count').rename(row_names)
count_ = int(summ_df.loc['2017', 'count'][0])
tab4 = ax.table(cellText=temp_df.values, rowLabels=temp_df.index, colLabels=temp_df.columns,
                colWidths=tab_col_widths, bbox=[0.25, .204, 0.75, .184])
width_ = tab4[1, -1].get_width()
height_ = tab4[0, 0].get_height()
tab4.add_cell(0, -1, width=width_, height=height_, text=f'2017 (count: {count_})', fontproperties=dict(size=tab_font_size))
tab4.auto_set_font_size(False)
tab4.set_fontsize(tab_font_size)
for cell_ in tab4.get_children():
    cell_._linewidth = tab_line_width

temp_df = summ_df.loc['All years'].drop(columns='count').rename(row_names)
count_ = int(summ_df.loc['All years', 'count'][0])
tab5 = ax.table(cellText=temp_df.values, rowLabels=temp_df.index, colLabels=temp_df.columns,
                colWidths=tab_col_widths, bbox=[0.25, 0, 0.75, .184])
width_ = tab5[1, -1].get_width()
height_ = tab5[0, 0].get_height()
tab5.add_cell(0, -1, width=width_, height=height_, text=f'All (count: {count_})', fontproperties=dict(size=tab_font_size))
tab5.auto_set_font_size(False)
tab5.set_fontsize(tab_font_size)
for cell_ in tab5.get_children():
    cell_._linewidth = tab_line_width

fig.savefig(os.path.join(common.FIGURES_DIR, 'Summary_stats.jpeg'), format="jpg", bbox_inches='tight', pad_inches=0.1, dpi=600)